# Visualize AFOG Attacks on SSD-300
This notebook contains code for attacking and visualizing a single image with AFOG on SSD-300. Note that all three regression-based detectors use VOC, not COCO, for evaluation. Please ensure you have followed all setup steps in README.md before running this notebook, and that you are using a Conda environment with requirements_reg.txt as the package list. 

In [ ]:
import os
import sys
sys.path.append(os.getcwd()[:os.getcwd().find("demos")])
from utils.dataset_utils.voc.preprocessing import letterbox_image_padded
from utils.misc_utils.visualization import visualize_detections
from keras import backend as K
from models.ssd import SSD300
from PIL import Image
from afog.attacks import *
import os
K.clear_session()
%load_ext autoreload
%autoreload 2

Prepare the weights file and load it into the model. Change the path to the weights as necessary.

In [ ]:
weights = '../model_files/SSD300.h5' 
detector = SSD300(weights=weights)

The three important hyperparameters for AFOG attacks are (i) the attack budget measured in L-inf norm, (ii) the attack learning rate, and (iii) the number of attack iterations.

In [ ]:
eps = 8 / 255.       # Hyperparameter: epsilon in L-inf norm
eps_iter = 2 / 255.  # Hyperparameter: attack learning rate
n_iter = 10          # Hyperparameter: number of attack iterations

First, visualize a single image with no attacks (benign prediction).

In [ ]:
fpath = "../datasets/VOCdevkit/VOC2007/JPEGImages/000002.jpg"

input_img = Image.open(fpath)
x_query, x_meta = letterbox_image_padded(input_img, size=detector.model_img_size)
detections_query = detector.detect(x_query, conf_threshold=detector.confidence_thresh_default)
visualize_detections({'Benign (No Attack)': (x_query, detections_query, detector.model_img_size, detector.classes)})

Now apply AFOG to the image and visualize the adversarial predictions. AFOG mode can be changed with the mode parameter to either "baseline", "vanishing", or "fabrication."

In [ ]:
# Generation of the adversarial example
x_adv_untargeted = afog_cnn(victim=detector, x_query=x_query, mode="baseline", n_iter=n_iter, eps=eps, eps_iter=eps_iter)

# Visualizing the detection results on the adversarial example and compare them with that on the benign input
detections_adv_untargeted = detector.detect(x_adv_untargeted, conf_threshold=detector.confidence_thresh_default)
visualize_detections({'Benign (No Attack)': (x_query, detections_query, detector.model_img_size, detector.classes),
                      'AFOG': (x_adv_untargeted, detections_adv_untargeted, detector.model_img_size, detector.classes)})